In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import psutil
NUM_WORKER = psutil.cpu_count(logical = False)

def Spark():
    """[summary]
    Returns:
        [type]: [description]
    """    
    conf_spark = SparkConf().set("spark.driver.host", "127.0.0.1")
    sc = SparkContext(conf = conf_spark)
    spark = SparkSession(sc)
    spark.conf.set("spark.sql.shuffle.partitions", NUM_WORKER)
    print('Spark UI address {}'.format(spark.sparkContext.uiWebUrl))
    return spark

sess = Spark()

Spark UI address http://127.0.0.1:4040


In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
sample = sess.read.csv("sample.csv",header=True)

In [4]:
movie = sess.read.csv("./ml-latest/movies.csv",header=True)

In [25]:
movie_copy = movie.withColumn("year",F.regexp_extract(movie.title,r"(\d{4})",0).cast(T.IntegerType()))
movie_copy = movie_copy.withColumn("genre",F.explode(F.split(movie.genres,pattern="\|")))
movie_copy = movie_copy.select("movieId","title","genre","year")
genres = movie_copy.select("genre").distinct().toPandas()['genre'].tolist()

In [6]:
sample_copy = sample.select("userId","movieId")

In [44]:
total = sample_copy.join(movie_copy,["movieId"],'left')


In [70]:
idx = []
for g in genres:
    movie_genre = total.where("genre == '"+g+"'")
    ids = movie_genre.groupby("movieId").count().sort("count",ascending=False)
    idx += list(ids.select("movieId").toPandas()['movieId'][:2])

In [93]:
popular = total.groupby("movieId").count().sort("count",ascending=False)
popular.select("movieId").limit(20).join(movie,["movieId"],'left').sample(False,.5).limit(5).show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|   4306|        Shrek (2001)|Adventure|Animati...|
|      1|    Toy Story (1995)|Adventure|Animati...|
|    356| Forrest Gump (1994)|Comedy|Drama|Roma...|
|    480|Jurassic Park (1993)|Action|Adventure|...|
|   1197|Princess Bride, T...|Action|Adventure|...|
+-------+--------------------+--------------------+



# cold start recommend()

In [94]:
def cold_start_recommend():
    movie_copy = movie.withColumn("year",F.regexp_extract(movie.title,r"(\d{4})",0).cast(T.IntegerType()))
    movie_copy = movie_copy.withColumn("genre",F.explode(F.split(movie.genres,pattern="\|")))
    movie_copy = movie_copy.select("movieId","title","genre","year")
    genres = movie_copy.select("genre").distinct().toPandas()['genre'].tolist()
    
    sample_copy = sample.select("userId","movieId")
    
    total = sample_copy.join(movie_copy,["movieId"],'left')
    popular = total.groupby("movieId").count().sort("count",ascending=False)
    return popular.select("movieId").limit(20).join(movie,["movieId"],'left').sample(False,.5).limit(5)

In [107]:
import time
start = time.time()

cold_start_recommend().show()

end = time.time()
print(end-start)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|    356| Forrest Gump (1994)|Comedy|Drama|Roma...|
|    480|Jurassic Park (1993)|Action|Adventure|...|
|   1197|Princess Bride, T...|Action|Adventure|...|
|    780|Independence Day ...|Action|Adventure|...|
|  79132|    Inception (2010)|Action|Crime|Dram...|
+-------+--------------------+--------------------+

2.322582960128784
